<a href="https://colab.research.google.com/github/jmhuer/quantum_dots/blob/main/quantum_dots_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data from excell


In [21]:
import pandas as pd
excell = pd.ExcelFile("/content/QDots data - Juan.xlsx")
excell.sheet_names
df = excell.parse("Sheet1")
df.columns = df.columns.map(str)
df = df.dropna().reset_index(drop=True)

# X = df[X_columns].dropna().reset_index(drop=True)
# Y1 = df[Y1_columns].dropna().reset_index(drop=True)
# Y2 = df[Y2_columns].dropna().reset_index(drop=True)
# .reshape(-1,1)

#need to make all vals 0-1 for better measurments of error

# Ploting utils

In [22]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()


## Pre-processing + model defenitions before training

In [80]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

X_columns  = ["415","445","480", "515", "555", "590", "630", "680"]
Y1_columns = ["# emitters λ1 (x1010)"]
Y2_columns = ["# emitters λ2 (x1010)"]
Y_columns  = ["# emitters λ1 (x1010)", "# emitters λ2 (x1010)"]

##LINEAR REGRESSION
lr_reg = make_pipeline(LinearRegression())

##RandomForestRegressor
rf_reg = make_pipeline(StandardScaler(), RandomForestRegressor())
parameters = {'randomforestregressor__max_depth':[1,2,3,4,5]}
rf_reg = GridSearchCV(rf_reg, parameters)
  
##SVM
svr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
parameters = {'svr__kernel':['linear', 'rbf'], 'svr__epsilon':[0.1, 0.2, 0.3,0.6], 'svr__C': [0.1,0.5,1,2,3]}
svr = GridSearchCV(svr, parameters)

##For Cross validations 
kf = KFold(n_splits=10, random_state=42, shuffle=True) # Define the split - into 10 folds 
k_accuracy = {"lr_reg": [], "rf_reg": [], "svr": []}


##Training loop w/ cross-validation 

In [81]:
for train_index, test_index in kf.split(df):
    X_train, X_test = df.loc[train_index, X_columns], df.loc[test_index, X_columns]
    Y_train, Y_test = df.loc[train_index, Y1_columns], df.loc[test_index, Y1_columns]

    ##LINEAR REGRESSION
    lr_reg.fit(X_train, Y_train.values.ravel())
    Y_pred = lr_reg.predict(X_test)
    k_accuracy['lr_reg'].append(mean_squared_error(Y_test, Y_pred))

    ##RandomForestRegressor
    rf_reg.fit(X_train, Y_train.values.ravel())
    Y_pred = rf_reg.predict(X_test)
    k_accuracy['rf_reg'].append(mean_squared_error(Y_test, Y_pred))

    ##Support Vecotor Regressor 
    svr.fit(X_train, Y_train.values.ravel())
    Y_pred = svr.predict(X_test)
    k_accuracy['svr'].append(mean_squared_error(Y_test, Y_pred))

    print(".")

for i in k_accuracy.keys():
    print(i,": ", sum(k_accuracy[i]) / len(k_accuracy[i]))

.
.
.
.
.
.
.
.
.
.
lr_reg :  4.050225300323953
rf_reg :  0.8540383684384855
svr :  4.476981793012247
